&emsp;&emsp;$k$近邻算法是机器学习中所有算法理论中最简单、最容易理解的算法。它是基本的分类与回归方法，它的输入为实例的特征向量，通过计算新数据与训练数据特征值之间的距离，然后选取$k$个距离最近的邻居进行分类判断(多数表决)或者回归。如果$k=1$,那么新数据被简单地分配给其近邻的类。<br>
&emsp;&emsp;对于分类问题：输出为实例的类别。分类时，对于新的实例，根据最近的$k$个实例的类别通过多数表决的方式进行预测。<br>
&emsp;&emsp;对于回归问题：输出为实例的值。回归时，对于新的实例，取其最近的$k$个实例的值的平均值最为预测值。<br>




In [2]:
import numpy as np
import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
import sklearn.neighbors as neighbors

In [3]:
digits=datasets.load_digits()

In [8]:
digits.data.shape,digits.target.shape


((1797, 64), (1797,))

In [9]:
x_train,x_test,y_train,y_test=model_selection.train_test_split(digits.data,digits.target,test_size=0.25,random_state=0,stratify=digits.target)
print('x_train shape:',x_train.shape)
print('y_train shape:',y_train.shape)

x_train shape: (1347, 64)
y_train shape: (1347,)


自己实现knn：暴力计算

In [10]:
def classify(x_train,y_train,k,x):
    sources=np.empty(y_train.shape)
    for i,e in enumerate(x_train):
        d=np.linalg.norm(e-x)
        sources[i]=d
    indices=np.argsort(sources)[:k]

    knn={}
    for i in indices:
        l=y_train[i]
        if l in knn:
            knn[l]+=1
        else:
            knn[l]=1
    return max(knn)

In [11]:
y_pred=np.empty(y_test.shape)
for i,e in enumerate(x_test):
    y_pred[i]=classify(x_train,y_train,3,e)
acc=metrics.classification_report(y_test,y_pred)
print(acc)

precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       0.98      0.96      0.97        46
           2       0.98      0.98      0.98        44
           3       0.93      0.93      0.93        46
           4       1.00      0.96      0.98        45
           5       0.98      0.98      0.98        46
           6       0.98      1.00      0.99        45
           7       1.00      1.00      1.00        45
           8       0.95      0.91      0.93        43
           9       0.88      0.96      0.91        45

    accuracy                           0.97       450
   macro avg       0.97      0.97      0.97       450
weighted avg       0.97      0.97      0.97       450



sklean 中实现了三种搜索方式, 分别是 BallTree, KDTree, brute-force. KDTree 是一颗二叉树, BallTree 是 KDTree 的优化版本, 而 brute-force 就是原始的暴力线性扫描. 这里要着重说明的是, KDTree 与 BallTree 虽然优化了搜索速度, 但牺牲了精确度. 因此对于小数据集(如 n < 30), 使用 brute-force 仍然是一个最佳选择.

In [12]:
nbrs=neighbors.KNeighborsClassifier(n_neighbors=3)
nbrs.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [13]:
y_pred=nbrs.predict(x_test)
acc=metrics.classification_report(y_test,y_pred)
print(acc)

precision    recall  f1-score   support

           0       1.00      1.00      1.00        45
           1       0.98      1.00      0.99        46
           2       1.00      1.00      1.00        44
           3       0.94      0.98      0.96        46
           4       1.00      0.98      0.99        45
           5       1.00      1.00      1.00        46
           6       1.00      1.00      1.00        45
           7       0.98      1.00      0.99        45
           8       1.00      0.95      0.98        43
           9       0.98      0.96      0.97        45

    accuracy                           0.99       450
   macro avg       0.99      0.99      0.99       450
weighted avg       0.99      0.99      0.99       450

